In [1]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings

In [2]:
!pip install maticalgos

In [3]:
!pip install maticalgos --upgrade
      

In [4]:
from maticalgos.historical import historical

ma = historical('saurav17@iiserbpr.ac.in')

In [5]:
ma.login("417113")

In [6]:
dates = ma.get_dates("nifty")

In [7]:
dates

['20190211',
 '20190212',
 '20190213',
 '20190214',
 '20190215',
 '20190218',
 '20190219',
 '20190220',
 '20190221',
 '20190222',
 '20190225',
 '20190226',
 '20190227',
 '20190228',
 '20190301',
 '20190305',
 '20190306',
 '20190307',
 '20190308',
 '20190311',
 '20190312',
 '20190313',
 '20190314',
 '20190315',
 '20190318',
 '20190319',
 '20190320',
 '20190322',
 '20190325',
 '20190326',
 '20190327',
 '20190328',
 '20190329',
 '20190401',
 '20190402',
 '20190403',
 '20190404',
 '20190405',
 '20190408',
 '20190409',
 '20190410',
 '20190411',
 '20190412',
 '20190415',
 '20190416',
 '20190418',
 '20190422',
 '20190423',
 '20190424',
 '20190425',
 '20190426',
 '20190430',
 '20190502',
 '20190503',
 '20190506',
 '20190507',
 '20190508',
 '20190509',
 '20190510',
 '20190513',
 '20190514',
 '20190515',
 '20190516',
 '20190517',
 '20190520',
 '20190521',
 '20190522',
 '20190523',
 '20190524',
 '20190527',
 '20190528',
 '20190529',
 '20190530',
 '20190531',
 '20190603',
 '20190604',
 '20190606',

In [8]:
trades = pd.DataFrame()

In [18]:
# Initialize an empty DataFrame to store trades
trades = pd.DataFrame(columns=["type", "sellprice", "symbol", "selltime", "qty", "date", "sl", "target", "pos", "strike"])

# Iterate over the time range from 9:30 to 2:30
for i in dates[-50:]:
    date = dt.datetime.strptime(i, "%Y%m%d").date()
    df = ma.get_data("nifty", date)
    df['datetime'] = df['date'] + " " + df['time']
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index(df['datetime'])
    df[['open', 'high', 'low', "close"]] = df[['open', 'high', 'low', "close"]].astype(float)
    spot_data = df[df['symbol'] == "NIFTY"]
    spot_data = spot_data[(spot_data['datetime'].dt.time >= pd.Timestamp('9:30').time()) &
                          (spot_data['datetime'].dt.time <= pd.Timestamp('14:30').time())]
    spot_data = spot_data.resample('15T').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'})
    spot_data['color'] = ['Green' if close > open else 'Red' for open, close in
                          zip(spot_data['open'], spot_data['close'])]
    exp = df.iloc[0]['symbol'][5:12]
    trades_list = []
    in_position = False
    
    # Iterate over each candle in spot_data
    for s in range(len(spot_data)):
        currentcandle = spot_data.iloc[s]
        
        try:
            if currentcandle.name not in df.index:
                print(f"Timestamp '{currentcandle.name}' not found in DataFrame. Skipping processing for this row.")
                continue  # Skip processing for this row
            
            if spot_data.loc[currentcandle.name, 'color'] == 'Red': 
                print("placing trade!")
                cst = currentcandle['open']
                cst = int(round(cst/50, 0) * 50)
                opdata = {"CE": df[df['symbol'] == "NIFTY" + exp + str(cst) + "CE"],
                          "PE": df[df['symbol'] == "NIFTY" + exp + str(cst) + "PE"]}
                ce_sellprice = opdata["CE"].loc[currentcandle.name]
                trade  = {"type": "CE", "sellprice": ce_sellprice["open"], "symbol": ce_sellprice['symbol'], 
                          "selltime": currentcandle.name.time(), "qty": 50, "date": currentcandle.name.date(), 
                          "sl": ce_sellprice["open"] + (ce_sellprice["open"] * 20 / 100),
                          "target": ce_sellprice["open"] - (ce_sellprice["open"] * 50 / 100), 
                          "pos": True, "strike": cst}
                trades_list.append(trade) 
                in_position = True
                
            elif spot_data.loc[currentcandle.name, 'color'] == 'Green':
                print("placing trade!")
                cst = currentcandle['open']
                cst = int(round(cst/50, 0) * 50)
                opdata = {"CE": df[df['symbol'] == "NIFTY" + exp + str(cst) + "CE"],
                          "PE": df[df['symbol'] == "NIFTY" + exp + str(cst) + "PE"]}
                pe_sellprice = opdata["PE"].loc[currentcandle.name]
                trade =  {"type": "PE", "sellprice": pe_sellprice["open"], "symbol": pe_sellprice['symbol'], 
                          "selltime": currentcandle.name.time(), "qty": 50, "date": currentcandle.name.date(), 
                          "sl": pe_sellprice["open"] + (pe_sellprice["open"] * 20 / 100),
                          "target": pe_sellprice["open"] - (pe_sellprice["open"] * 50 / 100), 
                          "pos": True, "strike": cst}
                trades_list.append(trade)
                in_position = True   
                
            if in_position:
                for trade in trades_list:
                    if opdata[trade["type"]].loc[currentcandle.name]['high'] >= trade['sl'] and trade['pos']:
                        trade['buyprice'] = trade['sl']
                        trade['buytime'] = currentcandle.name.time()
                        trade['reason'] = "SL HIT"
                        trade['pnl'] = (trade['sellprice'] - trade['buyprice']) * trade['qty']
                        trade['pos'] = False
                        trades = trades.append(trade, ignore_index=True)
                        
                    if opdata[trade["type"]].loc[currentcandle.name]['low'] <= trade['target'] and trade['pos']:
                        trade['buyprice'] = trade['target']
                        trade['buytime'] = currentcandle.name.time()
                        trade['reason'] = "TARGET HIT"
                        trade['pnl'] = (trade['sellprice'] - trade['buyprice']) * trade['qty']
                        trade['pos'] = False
                        trades = trades.append(trade, ignore_index=True)
                    
                    if currentcandle.name.time() == dt.time(14, 55) and trade['pos']: 
                        trade['buyprice'] = opdata[trade["type"]].loc[currentcandle.name]['open']
                        trade['buytime'] = currentcandle.name.time()
                        trade['reason'] = "Time Up"
                        trade['pnl'] = (trade['sellprice'] - trade['buyprice']) * trade['qty']
                        trade['pos'] = False
                        trades = trades.append(trade, ignore_index=True)
        
        except KeyError as e:
            print(f"KeyError: {e} - Timestamp '{currentcandle.name}' not found in DataFrame.")
            continue  # Skip processing for this row


placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!


/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be remo

placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!


/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be remo

placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!


...

/tmp/ipykernel_16932/2519492224.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)


placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!


/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)


placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!
placing trade!


/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append(trade, ignore_index=True)
/tmp/ipykernel_16932/2519492224.py:69: FutureWarning: The frame.append method is deprecated and will be remo

In [20]:
trades.head()

,type,sellprice,symbol,selltime,qty,date,sl,target,pos,strike,buyprice,buytime,pnl,reason
0,CE,90.40,NIFTY23NOV2319750CE,09:30:00,50,2023-11-20,108.48,45.200,False,19750,108.48,09:45:00,-904.0,SL HIT
1,PE,67.60,NIFTY23NOV2319700PE,10:00:00,50,2023-11-20,81.12,33.800,False,19700,81.12,11:15:00,-676.0,SL HIT
2,PE,68.05,NIFTY23NOV2319700PE,10:15:00,50,2023-11-20,81.66,34.025,False,19700,81.66,11:15:00,-680.5,SL HIT
3,PE,75.35,NIFTY23NOV2319750PE,10:45:00,50,2023-11-20,90.42,37.675,False,19750,90.42,11:15:00,-753.5,SL HIT
4,CE,83.40,NIFTY23NOV2319700CE,11:30:00,50,2023-11-20,100.08,41.700,False,19700,100.08,13:15:00,-834.0,SL HIT


,sellprice,sl,target,buyprice,pnl
count,32.000000,32.000000,32.000000,32.000000,32.000000
mean,68.081250,81.697500,34.040625,64.631719,172.476562
std,34.168156,41.001787,17.084078,47.230095,1208.018576
min,8.350000,10.020000,4.175000,8.750000,-1329.500000
25%,44.050000,52.860000,22.025000,27.663750,-955.500000
50%,65.525000,78.630000,32.762500,37.707500,-236.250000
75%,97.675000,117.210000,48.837500,114.660000,1340.937500
max,132.950000,159.540000,66.475000,159.540000,2882.500000
